In [ ]:
pip install selenium webdriver-manager


In [ ]:
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager




In [ ]:
#Config variable declaration

MY_NAME = "Sayandip"        # your name
MY_SLOT = "10:00-11:00"     # the slot you want
GROUP_NAME = "Test1"   # group name
POLL_SECONDS = 5            # check every X seconds
PERSIST_LOGIN = True
USER_DATA_DIR = "wpp_profile"

In [ ]:
# Helper classes defined

def robust_find(driver, locator, timeout=30):
    return WebDriverWait(driver, timeout).until(EC.presence_of_element_located(locator))

def handle_possible_popup(driver):
    try:
        popup = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, "//div[@role='dialog']"))
        )
        print(" Detected popup. Closing it…")
        try:
            btn = popup.find_element(By.XPATH, ".//button")
            btn.click()
        except Exception:
            body = driver.find_element(By.TAG_NAME, "body")
            body.send_keys(Keys.ESCAPE)
    except Exception:
        pass

In [ ]:
#function to get into whatsapp and enter the group 

def open_whatsapp_and_go_to_group(group_name: str) -> webdriver.Chrome:
    chrome_options = Options()
    if PERSIST_LOGIN:
        chrome_options.add_argument(f"--user-data-dir={USER_DATA_DIR}")
    chrome_options.add_argument("--profile-directory=Default")
    chrome_options.add_argument("--window-size=1200,900")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get("https://web.whatsapp.com/")
    print("Opened WhatsApp Web. If needed, scan QR.")

    WebDriverWait(driver, 40).until(
        EC.presence_of_element_located((By.XPATH, "//div[@role='grid']"))
    )
    handle_possible_popup(driver)

    # Search for group
    search = robust_find(driver, (By.XPATH, "//*[@id='side']//div[@contenteditable='true' and @role='textbox']"), timeout=30)
    search.click()
    search.send_keys(Keys.CONTROL, "a")
    search.send_keys(Keys.DELETE)
    search.send_keys(group_name)
    time.sleep(1)
    search.send_keys(Keys.ENTER)

    robust_find(driver, (By.XPATH, f"//header//*[text()='{group_name}' or @title='{group_name}']"), timeout=20)
    print(f" Entered group: {group_name}")
    return driver

In [ ]:
#function to capture the last message on the group

def get_last_message_text(driver) -> str:
    try:
        bubbles = driver.find_elements(By.XPATH, "//div[@data-id='true' or contains(@class,'copyable-text')]")
        if not bubbles:
            return ""
        last_bubble = bubbles[-1]
        text = driver.execute_script("return arguments[0].innerText;", last_bubble)
        return text.strip()
    except Exception:
        return ""

In [ ]:
def get_compose(driver):
    xpaths = [
        "//footer//div[@contenteditable='true']",
        "//div[@contenteditable='true' and @role='textbox']",
        "//div[@contenteditable='true']",
    ]
    for xp in xpaths:
        try:
            el = driver.find_element(By.XPATH, xp)
            if el:
                return el
        except Exception:
            continue
    raise RuntimeError("Compose box not found. UI may have changed.")

import pyperclip
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
def send_message(driver, message: str):
    compose = get_compose(driver)
    compose.click()
    time.sleep(0.3)

    # Copy to clipboard
    pyperclip.copy(message)

    # On Mac → CMD+V, on Windows/Linux → CTRL+V
    actions = ActionChains(driver)
    actions.key_down(Keys.COMMAND).send_keys("v").key_up(Keys.COMMAND).perform()

    time.sleep(0.3)
    compose.send_keys(Keys.ENTER)
    print(" Sent updated schedule in ONE single WhatsApp message with formatting & emojis preserved.")

In [ ]:
# =============== CORE LOGIC ===============
def is_probable_schedule(text: str) -> bool:
    """Check if text looks like a schedule message more flexibly."""
    lines = [ln for ln in text.splitlines() if ln.strip()]
    if len(lines) < 10:
        return False
    has_first = any("00:00-01:00" in ln for ln in lines)
    has_last = any("23:00-00:00" in ln for ln in lines)
    return has_first and has_last

In [ ]:
def fill_my_slot(message: str, my_slot: str, my_name: str) -> (str, bool):
    """
    Replace ONLY the target slot line if it's empty.
    Remove unnecessary blank lines.
    Wrap any date lines with exclamation emojis.
    """
    out_lines = []
    replaced = False

    for line in message.splitlines():
        stripped = line.strip()

        # Wrap date lines with exclamations
        if re.match(r"^Date\s*:.*\d{2}/\d{2}/\d{4}", stripped) or re.match(r"^\d{2}/\d{2}/\d{4}$", stripped):
            line = f"‼️ {stripped.replace('Date:', '').strip()} ‼️"

        # Slot replacement
        if stripped.startswith(my_slot):
            m = re.match(rf"^(\s*{re.escape(my_slot)}\s*-\s*)(.*)$", line)
            if m:
                prefix = m.group(1)
                after = m.group(2)
                if after.strip() == "" or after.strip().lower() in {"-", "na", "tbd", "nil"}:
                    line = f"{prefix}{my_name}"
                    replaced = True

        # Skip blank lines
        if line.strip():
            out_lines.append(line)

    return "\n".join(out_lines), replaced

In [ ]:
# =============== MAIN LOOP ===============
def main():
    driver = open_whatsapp_and_go_to_group(GROUP_NAME)
    last_seen_signature = None
    print(" Watching for new schedule messages...")

    while True:
        msg_text = get_last_message_text(driver)
        if not msg_text:
            time.sleep(POLL_SECONDS)
            continue

        signature = (len(msg_text), msg_text[:50])
        if signature == last_seen_signature:
            time.sleep(POLL_SECONDS)
            continue

        if is_probable_schedule(msg_text):
            print(" Detected a timetable message.")
            updated, replaced = fill_my_slot(msg_text, MY_SLOT, MY_NAME)
            if replaced:
                print(f"Slot {MY_SLOT} is empty. Filling it with {MY_NAME}...")
                send_message(driver, updated)
            else:
                print(" Slot already filled, not sending.")
        else:
            print(" Last message is not a timetable.")

        last_seen_signature = signature
        time.sleep(POLL_SECONDS)

if __name__ == "__main__":
    main()